In [28]:
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import mean_squared_error, accuracy_score
import os
import statistics
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", None)

In [5]:
benign_datas = pd.read_pickle('/home/sdsra/Downloads/CTU-13-Dataset/benign/datas-final-by-origin-tool.pkl')

In [6]:
benign_datas['is_malware'] = 0

In [7]:
malware_datas = pd.read_pickle('/home/sdsra/Downloads/CTU-13-Dataset/malware/1/datas-by-origin-tool.pkl')
for i in [2,3,4,5,6,7,8,9,10,11,12,13,42,43,44,45,46,47,48,49,50,52,53,54]:
    pkl_path = '/home/sdsra/Downloads/CTU-13-Dataset/malware/{}/datas-by-origin-tool.pkl'.format(str(i))
    if os.path.exists(pkl_path):
        tmp_data = pd.read_pickle(pkl_path)
        malware_datas = pd.concat([malware_datas, tmp_data], ignore_index=True)

In [8]:
len(benign_datas), len(malware_datas)

(6778, 968)

In [9]:
# malware_datas = malware_datas[malware_datas['label'] == 'botnet']
upsampled_malware_datas = malware_datas.sample(len(benign_datas), replace=True)

In [10]:
upsampled_malware_datas['is_malware'] = 1

In [11]:
datas = pd.concat([benign_datas, upsampled_malware_datas], ignore_index=True)

/home/sdsra/anaconda3/envs/gep-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [12]:
numeric_columns = list(filter(lambda x: x not in ['src_ip','src_port','dst_ip','dst_port','proto'], benign_datas.columns))

In [13]:
datas['is_malware'].value_counts()

1    6778
0    6778
Name: is_malware, dtype: int64

In [14]:
X_train, X_test, y_train, y_test = train_test_split(datas, datas['is_malware'], test_size=0.33, random_state=42)

# cross validation

In [34]:
cls = SVC(gamma='scale')

In [39]:
score = cross_validate(cls, datas[numeric_columns], datas['is_malware'], cv=10, return_train_score=False)

In [40]:
np.mean(score['test_score'])

0.9336815858616226

## Fit SVM

In [ ]:
cls = SVC(gamma='scale')

In [17]:
cls.fit(X_train[numeric_columns], y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [24]:
score = accuracy_score(y_test, cls.predict(X_test[numeric_columns]))

In [25]:
score

0.915958873491283

In [26]:
X_test['predict_score'] = cls.predict(X_test[numeric_columns])

/home/sdsra/anaconda3/envs/gep-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
X_test[['is_malware', 'predict_score']]

,is_malware,predict_score
811,0,0
11412,1,1
2802,0,0
4029,0,0
10311,1,1
10371,1,1
10952,1,0
10935,1,1
1373,0,0
7153,1,0


# cross validation

In [31]:
score = cross_validate(cls, datas[numeric_columns], datas['is_malware'], cv=10)

In [32]:
np.mean(score.tr

/home/sdsra/anaconda3/envs/gep-gpu/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([3.78950334, 3.66413546, 3.73150611, 3.76993895, 3.72476006,
        3.7133131 , 3.63252664, 3.71340442, 3.57251334, 3.4778614 ]),
 'score_time': array([0.26471496, 0.2608273 , 0.26063824, 0.26516294, 0.26541424,
        0.2630012 , 0.2576015 , 0.26457024, 0.25557137, 0.25005054]),
 'test_score': array([0.94026549, 0.92477876, 0.93362832, 0.94616519, 0.93879056,
        0.9380531 , 0.92625369, 0.92772861, 0.93426883, 0.92688331]),
 'train_score': array([0.93786885, 0.93942623, 0.93959016, 0.93704918, 0.93655738,
        0.93672131, 0.93909836, 0.93860656, 0.94082937, 0.94336994])}